<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/Learning_TextGrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU textgrad --progress-bar off
!pip install -qU openai --progress-bar off

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [5]:
# @title load fireworks API key
#connect to google drive
from google.colab import drive
import json
import os


drive.mount('/content/drive')

with open('/content/drive/MyDrive/env/env.json') as jsonfile:
    env = json.load(jsonfile)

os.environ['OPENAI_API_KEY'] = env['fireworks.ai']['apiKey']
os.environ['OPENAI_BASE_URL'] = "https://api.fireworks.ai/inference/v1"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from openai import OpenAI

from textgrad.engine import get_engine
from textgrad import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.loss import TextLoss
from textgrad.engine.openai import ChatOpenAI

engine = ChatOpenAI(model_string='accounts/fireworks/models/mixtral-8x7b-instruct')

In [9]:
x = Variable("A sntence with a typo", role_description="The input sentence", requires_grad=True)
engine.generate("Hello how are you?")

"Hello! I'm an artificial intelligence, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?"

## Introduction: Loss

Again, Loss in TextGrad is the metaphorical equivalent of loss in PyTorch. We use Losses in different form in TextGrad but for now we will focus on a simple TextLoss. TextLoss is going to evaluate the loss wrt a string.

In [10]:
system_prompt = Variable("Evaluate the correctness of this sentence", role_description="The system prompt")
loss = TextLoss(system_prompt, engine=engine)

## Introduction: Optimizer

Keeping on the analogy with PyTorch, the optimizer in TextGrad is the object that will update the parameters of the model. In this case, the parameters are the variables that have `requires_grad` set to `True`.

**NOTE** This is a text optimizer! It will do all operations with text!

In [11]:
optimizer = TextualGradientDescent(parameters=[x], engine=engine)

## Putting it all together

We can now put all the pieces together. We have a variable, an engine, a loss, and an optimizer. We can now run a single optimization step.

In [12]:
l = loss(x)
l.backward(engine)
optimizer.step()

INFO:textgrad:LLMCall function forward
INFO:textgrad:_backward_through_llm prompt
INFO:textgrad:_backward_through_llm gradient
INFO:textgrad:_backward_through_llm prompt
INFO:textgrad:_backward_through_llm gradient
INFO:textgrad:TextualGradientDescent prompt for update
INFO:textgrad:TextualGradientDescent optimizer response
INFO:textgrad:TextualGradientDescent updated text


In [13]:
x.value

'A sentence with a typo'

While here it is not going to be useful, we can also do multiple optimization steps in a loop! Do not forget to reset the gradients after each step!

In [ ]:
optimizer.zero_grad()